In [120]:
def ltr_extract():
    import Bio.SeqIO as SeqIO
    genome_data = {}
    with open('./napus.fna') as f:
        for record in SeqIO.parse(f, 'fasta'):
            genome_data[str(record.id).replace('.', '_')] = record.seq
    
    if os.path.exists("./extract"):
        for file in os.listdir('./extract'):
            os.remove(f'./extract/{file}')
    else: 
        os.makedir('./extract')
        
    with open('./napus.LTR.gff3', 'r') as g:
        cnt = 0
        for i, line in enumerate(g):
            if line.startswith("##"):
                cnt+=1
            if not line.startswith("#") and line.split('\t')[2].strip() == 'long_terminal_repeat':
                start = int(line.split('\t')[3].strip())
                end = int(line.split('\t')[4].strip())
                name = line.split("\t")[0].strip()
                with open(f'./extract/{cnt}.fa', 'a')as h:
                    h.write(f'>{cnt}_{i%2}\n{genome_data[name][start:end+1].upper()}\n')

    del genome_data

In [121]:
ltr_extract()

In [123]:
!cat extract/6505.fa

>6505_0
GTAAGGTTTAAACGTCGATATACTATGTTAAACCTTCGTTCCTTTAGCTAGAATTCGTTTAGCTATGTTACTTGATCGATTAACCTCAATACTAACTTGTGTTCTGTTTGTTTGTATGTAGGAGATAAGACACAGCAAGTTTGTTTACCCAGTTCCCGTGAAGGTACGTCTGGGGGAAGAACAGAGTCTTCCAATTCACTAACTCAAAGAGAAAGAATTAATAAAAGGGTTGTGACCTTAAACTTATAAACGCCAAAGCTTAATCAAGAGCTAAACACCTAGCTCTCTTCTTCTGATTGAACTTAGCCTTCAGGTAAACACAAACGATCTCTCAACCAAGACAATTTGCTCTTTCACCTGAACCTCTCAATCTGTTGTTGTTGCCCCACCCTTCAAGTCACCAACGCGATCTTCTTTTATACTCAGGTTTTATGAAACAAAACCTAGAAGCTGACGTGGGCCACAGTACAACAAATACTCTGGAGGCCTGTTCCACTTCTCTGTGGGCTTCAGACACTTCAGCCCAAGTTCAAGATGGAGACAACAAACCCGACAA
>6505_1
GTAAGGTTTAAACGTCGATATACTATGTTAAACCTTCGTTCCTTTAGCTAGAATTCGTTTAGCTATGTTACTTGATCGATTAACCTCAATACTAACTTGTGTTCTGTTTGTTTGTATGTAGGAGATAAGACACAGCAAGTTTGTTTACCCAGTTCCCGTGAAGGTACGTCTGGGGGAAGAACAGAGTCTTCCAATTCACTAACTCAAAGAGAAAGAATTAATAAAAGGGTTGTGACCTTAAACTTATAAACGCCAAAGCTTAATCAAGAGCTAAACACCTAGCTCTCTTCTTCTGATTGAACTTAGCCTTCAGGTAAACACAAACGATCTCTCAACCAAGACAATTTGCTCTTTCACCTGAACCTCTCAATCTGTTGTTGTTGCCCCACCCTTCAAGTCACCAACGCGATCTTCTTTTATACTCAGG

In [124]:
from Bio.Align.Applications import MafftCommandline
import os

if os.path.exists("./aligned"):
    for file in os.listdir('./aligned'):
        os.remove(f'./aligned/{file}')
else: 
    os.makedir('./aligned')
        
for filename in os.listdir('./extract'):
    mafft_cline = MafftCommandline(input=f"./extract/{filename}", thread=12)
    stdout, stderr = mafft_cline()
    with open(f'./aligned/aligned_{filename}', 'w') as f:
        f.write(stdout)

In [139]:
from cogent3 import get_model, load_aligned_seqs
from cogent3.evolve import distance

aln = load_aligned_seqs('aligned/aligned_270.fa', moltype="dna")
d = distance.EstimateDistances(aln, submodel=get_model("K80"))
d.run(show_progress=False)
dists = d.get_pairwise_distances()
dists.array[0][1]

0.11535547456537795

In [ ]:
from cogent3.app import evo, io

loader = io.load_aligned(format="fasta", moltype="dna")

for filename in os.listdir('./aligned'):
    aln = loader(f'aligned/{filename}')
    model = evo.model("K80", tree="data/primate_brca1.tree", optimise_motif_probs=True)
result = model(aln)
result
